In [ ]:
import pandas as pd
import json
import sqlite3
from google.colab import files

# --- 1. Load JSON (User Data) ---
print("Loading JSON data...")
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)

# --- 2. Load CSV (Order Data) ---
print("Loading CSV data...")
orders_df = pd.read_csv('orders.csv')

# --- 3. Load SQL (Restaurant Data) ---
print("Processing SQL data...")
# Create an in-memory database
conn = sqlite3.connect(':memory:')
# Read and execute your .sql file
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)
# Pull the data into a Pandas DataFrame
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)

# --- 4. Merge the Datasets (Left Join) ---
print("Merging datasets...")
# First Join: Orders + Users
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Second Join: Add Restaurant info (Cuisine and Rating)
# We drop restaurant_name from the SQL side to avoid duplicate columns
final_df = pd.merge(
    merged_df,
    restaurants_df[['restaurant_id', 'cuisine', 'rating']],
    on='restaurant_id',
    how='left'
)

# --- 5. Save and Download ---
file_name = 'final_food_delivery_dataset.csv'
final_df.to_csv(file_name, index=False)

print(f"Success! '{file_name}' has been created.")

# This line triggers an automatic download to your computer
files.download(file_name)

Loading JSON data...
Loading CSV data...
Processing SQL data...
Merging datasets...
Success! 'final_food_delivery_dataset.csv' has been created.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 1. Filter the dataset for Gold members only
gold_data = final_df[final_df['membership'] == 'Gold']

# 2. Group by city and sum the total_amount
city_revenue = gold_data.groupby('city')['total_amount'].sum().sort_values(ascending=False)

# 3. Display the result
print("Total Revenue from Gold Members by City:")
print(city_revenue)

print(f"\nThe city with the highest Gold member revenue is: {city_revenue.idxmax()}")

Total Revenue from Gold Members by City:
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

The city with the highest Gold member revenue is: Chennai


In [ ]:
# 1. Group by cuisine and calculate the mean (average) of total_amount
cuisine_aov = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

# 2. Display the result
print("Average Order Value (AOV) by Cuisine:")
print(cuisine_aov)

# 3. Identify the top cuisine
top_cuisine = cuisine_aov.idxmax()
print(f"\nThe cuisine with the highest Average Order Value is: {top_cuisine}")

Average Order Value (AOV) by Cuisine:
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

The cuisine with the highest Average Order Value is: Mexican


In [ ]:
# 1. Group by user_id and sum their total_amount
user_total_spending = final_df.groupby('user_id')['total_amount'].sum()

# 2. Filter for users who spent more than 1000
high_spenders = user_total_spending[user_total_spending > 1000]

# 3. Count how many users meet this criteria
distinct_user_count = len(high_spenders)

print(f"Number of distinct users with total spending > ₹1000: {distinct_user_count}")

Number of distinct users with total spending > ₹1000: 2544


In [ ]:
# 1. Calculate the total sum of all orders for every distinct user
user_total_spend = final_df.groupby('user_id')['total_amount'].sum()

# 2. Define the spending ranges (bins) and the specific labels you requested
bins = [0, 500, 1000, 2000, float('inf')]
labels = ['< 500', '500 – 1000', '1000 – 2000', '> 2000']

# 3. Categorize each user into a bucket
user_segments = pd.cut(user_total_spend, bins=bins, labels=labels, right=False)

# 4. Count how many distinct users fall into each category
category_counts = user_segments.value_counts().reindex(labels)

# 5. Print the final output
print("Distribution of Distinct Users by Total Spending:")
for range_label, count in category_counts.items():
    print(f"{range_label}: {count} users")

Distribution of Distinct Users by Total Spending:
< 500: 114 users
500 – 1000: 225 users
1000 – 2000: 699 users
> 2000: 1845 users


In [ ]:
# 1. Define the specific bins and labels
# We use 2.99 as the start to ensure 3.0 is included in the first bucket
bins = [2.99, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

# 2. Categorize the orders
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

# 3. Sum the revenue for each range
rating_range_revenue = final_df.groupby('rating_range')['total_amount'].sum()

# 4. Display the output
print("Total Revenue by Rating Range:")
for range_label, revenue in rating_range_revenue.items():
    print(f"{range_label}: ₹{revenue:,.2f}")

print(f"\nThe range with the highest revenue is: {rating_range_revenue.idxmax()}")

Total Revenue by Rating Range:
3.0 – 3.5: ₹2,136,772.70
3.6 – 4.0: ₹1,717,494.41
4.1 – 4.5: ₹1,960,326.26
4.6 – 5.0: ₹2,197,030.75

The range with the highest revenue is: 4.6 – 5.0


/tmp/ipython-input-1866610478.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_range_revenue = final_df.groupby('rating_range')['total_amount'].sum()


In [ ]:
# 1. Filter for Gold members only
gold_members_df = final_df[final_df['membership'] == 'Gold']

# 2. Group by city and calculate the mean (average) of total_amount
gold_city_aov = gold_members_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)

# 3. Display the result
print("Average Order Value (AOV) for Gold Members by City:")
for city, aov in gold_city_aov.items():
    print(f"{city}: ₹{aov:.2f}")

top_gold_city = gold_city_aov.idxmax()
print(f"\nThe city with the highest Average Order Value among Gold members is: {top_gold_city}")

Average Order Value (AOV) for Gold Members by City:
Chennai: ₹808.46
Hyderabad: ₹806.42
Bangalore: ₹793.22
Pune: ₹781.16

The city with the highest Average Order Value among Gold members is: Chennai


In [ ]:
# 1. Count distinct restaurants per cuisine
rest_counts = final_df.groupby('cuisine')['restaurant_id'].nunique()

# 2. Sum total revenue per cuisine
cuisine_revenue = final_df.groupby('cuisine')['total_amount'].sum()

# 3. Combine into a summary table
cuisine_analysis = pd.DataFrame({
    'Distinct Restaurants': rest_counts,
    'Total Revenue': cuisine_revenue
})

# 4. Calculate 'Revenue Per Restaurant' to find the most "efficient" cuisine
cuisine_analysis['Revenue Per Restaurant'] = cuisine_analysis['Total Revenue'] / cuisine_analysis['Distinct Restaurants']

# Sort by number of restaurants (ascending) to see the rarest cuisines first
cuisine_analysis = cuisine_analysis.sort_values(by='Distinct Restaurants', ascending=True)

# 5. Display the result
print("Cuisine Analysis: Restaurant Scarcity vs. Revenue")
print(cuisine_analysis)

# 6. Identify the winner (Lowest count with highest revenue per restaurant)
top_efficient = cuisine_analysis.sort_values(by='Revenue Per Restaurant', ascending=False).index[0]
print(f"\nThe most 'efficient' cuisine (highest revenue per restaurant) is: {top_efficient}")

Cuisine Analysis: Restaurant Scarcity vs. Revenue
         Distinct Restaurants  Total Revenue  Revenue Per Restaurant
cuisine                                                             
Chinese                   120     1930504.65            16087.538750
Indian                    126     1971412.58            15646.131587
Italian                   126     2024203.80            16065.109524
Mexican                   128     2085503.09            16292.992891

The most 'efficient' cuisine (highest revenue per restaurant) is: Mexican


In [ ]:
# 1. Calculate the total number of orders
total_orders = len(final_df)

# 2. Count the orders placed by Gold members
gold_orders_count = len(final_df[final_df['membership'] == 'Gold'])

# 3. Calculate the percentage
gold_percentage = (gold_orders_count / total_orders) * 100

# 4. Display the result rounded to the nearest integer
print(f"Percentage of total orders placed by Gold members: {round(gold_percentage)}%")

Percentage of total orders placed by Gold members: 50%


In [ ]:
# 1. Calculate Average Order Value (mean) and Order Count (size) for each restaurant
restaurant_stats = final_df.groupby(['restaurant_id', 'restaurant_name'])['total_amount'].agg(['mean', 'count'])

# 2. Filter for restaurants with less than 20 orders
low_volume_restaurants = restaurant_stats[restaurant_stats['count'] < 20]

# 3. Sort by the average value (mean) in descending order
top_aov_low_volume = low_volume_restaurants.sort_values(by='mean', ascending=False)

# 4. Display the result
if not top_aov_low_volume.empty:
    top_row = top_aov_low_volume.head(1)
    print("Top Restaurant with < 20 orders and highest AOV:")
    print(top_row)
else:
    print("No restaurants found with less than 20 orders.")

Top Restaurant with < 20 orders and highest AOV:
                                               mean  count
restaurant_id restaurant_name                             
294           Hotel Dhaba Multicuisine  1040.222308     13


In [ ]:
# 1. Define the specific list of options
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

# 2. Filter the main dataset for these 4 restaurants
filtered_df = final_df[final_df['restaurant_name'].isin(options)]

# 3. Calculate Average (mean) and Count
stats = filtered_df.groupby('restaurant_name')['total_amount'].agg(['mean', 'count'])

# 4. Filter for those with less than 20 orders
result = stats[stats['count'] < 20]

# 5. Sort by average value (mean) descending
winner = result.sort_values(by='mean', ascending=False)

print("Analysis of the specific options provided:")
print(winner)

if not winner.empty:
    print(f"\nTHE CORRECT ANSWER IS: {winner.index[0]}")
else:
    print("\nNone of these restaurants have less than 20 orders based on your data.")

Analysis of the specific options provided:
                           mean  count
restaurant_name                       
Ruchi Foods Chinese  686.603158     19

THE CORRECT ANSWER IS: Ruchi Foods Chinese


In [ ]:
# 1. Combination: City and Cuisine (Most common answer)
city_cuisine = final_df.groupby(['city', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)

# 2. Combination: City and Membership
city_member = final_df.groupby(['city', 'membership'])['total_amount'].sum().sort_values(ascending=False)

# 3. Combination: Cuisine and Membership
cuisine_member = final_df.groupby(['cuisine', 'membership'])['total_amount'].sum().sort_values(ascending=False)

print("--- Top 5 City + Cuisine Combinations ---")
print(city_cuisine.head(5))

print("\n--- Top 5 City + Membership Combinations ---")
print(city_member.head(5))

print("\n--- Top 5 Cuisine + Membership Combinations ---")
print(cuisine_member.head(5))

# Identify the absolute highest revenue combination (City + Cuisine)
top_pair = city_cuisine.index[0]
top_val = city_cuisine.values[0]
print(f"\nTHE HIGHEST REVENUE COMBINATION IS: {top_pair[0]} and {top_pair[1]} (₹{top_val:,.2f})")

--- Top 5 City + Cuisine Combinations ---
city       cuisine
Bangalore  Mexican    571004.61
           Italian    567881.80
           Indian     543014.04
Chennai    Mexican    534804.60
Bangalore  Chinese    525046.13
Name: total_amount, dtype: float64

--- Top 5 City + Membership Combinations ---
city       membership
Bangalore  Regular       1212243.99
Chennai    Gold          1080909.79
Pune       Gold          1003012.32
Bangalore  Gold           994702.59
Hyderabad  Regular        992626.39
Name: total_amount, dtype: float64

--- Top 5 Cuisine + Membership Combinations ---
cuisine  membership
Mexican  Regular       1072943.30
Italian  Regular       1018424.75
Mexican  Gold          1012559.79
Italian  Gold          1005779.05
Indian   Regular        992100.27
Name: total_amount, dtype: float64

THE HIGHEST REVENUE COMBINATION IS: Bangalore and Mexican (₹571,004.61)


In [ ]:
# Check only the provided options
check_list = [
    ('Indian', 'Gold'),
    ('Italian', 'Gold'),
    ('Indian', 'Regular'),
    ('Chinese', 'Regular')
]

# Calculate revenue for these specific pairs
results = {}
for cuis, memb in check_list:
    val = final_df[(final_df['cuisine'] == cuis) & (final_df['membership'] == memb)]['total_amount'].sum()
    results[f"{memb} + {cuis}"] = val

# Sort and print
for option, revenue in sorted(results.items(), key=lambda item: item[1], reverse=True):
    print(f"{option}: ₹{revenue:,.2f}")

Gold + Italian: ₹1,005,779.05
Regular + Indian: ₹992,100.27
Gold + Indian: ₹979,312.31
Regular + Chinese: ₹952,790.91


In [ ]:
# 1. Convert order_date to datetime objects
# Note: format is Day-Month-Year based on your CSV snippet
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# 2. Extract the quarter (1, 2, 3, or 4)
final_df['quarter'] = final_df['order_date'].dt.quarter

# 3. Group by quarter and sum the total_amount
quarterly_revenue = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

# 4. Map the numbers to "Q" labels for better readability
quarter_labels = {1: "Q1 (Jan-Mar)", 2: "Q2 (Apr-Jun)", 3: "Q3 (Jul-Sep)", 4: "Q4 (Oct-Dec)"}

print("Total Revenue by Quarter:")
for q, rev in quarterly_revenue.items():
    print(f"{quarter_labels[q]}: ₹{rev:,.2f}")

highest_q = quarterly_revenue.idxmax()
print(f"\nTHE HIGHEST REVENUE WAS GENERATED IN: {quarter_labels[highest_q]}")

Total Revenue by Quarter:
Q3 (Jul-Sep): ₹2,037,385.10
Q4 (Oct-Dec): ₹2,018,263.66
Q1 (Jan-Mar): ₹2,010,626.64
Q2 (Apr-Jun): ₹1,945,348.72

THE HIGHEST REVENUE WAS GENERATED IN: Q3 (Jul-Sep)


In [ ]:
# 1. Count the number of rows where membership is 'Gold'
gold_orders_total = len(final_df[final_df['membership'] == 'Gold'])

# 2. Display the result
print(f"Total number of orders placed by Gold members: {gold_orders_total}")

Total number of orders placed by Gold members: 4987


In [ ]:
# 1. Filter the dataset for orders in Hyderabad
hyderabad_orders = final_df[final_df['city'] == 'Hyderabad']

# 2. Sum the total_amount
total_revenue_hyd = hyderabad_orders['total_amount'].sum()

# 3. Print the result rounded to the nearest integer
print(f"Total revenue from Hyderabad: ₹{round(total_revenue_hyd)}")

Total revenue from Hyderabad: ₹1889367


In [ ]:
# 1. Count the number of unique user IDs in the dataset
distinct_users_count = final_df['user_id'].nunique()

# 2. Display the result
print(f"Number of distinct users who placed at least one order: {distinct_users_count}")

Number of distinct users who placed at least one order: 2883


In [ ]:
# 1. Filter the dataset for Gold members
gold_orders = final_df[final_df['membership'] == 'Gold']

# 2. Calculate the mean of total_amount
gold_aov = gold_orders['total_amount'].mean()

# 3. Display the result rounded to 2 decimal places
print(f"Average Order Value for Gold members: ₹{gold_aov:.2f}")

Average Order Value for Gold members: ₹797.15


In [ ]:
# 1. Filter the dataset for ratings 4.5 and above
high_rated_orders = final_df[final_df['rating'] >= 4.5]

# 2. Count the number of orders
order_count = len(high_rated_orders)

# 3. Display the result
print(f"Total orders placed for restaurants with rating ≥ 4.5: {order_count}")

Total orders placed for restaurants with rating ≥ 4.5: 3374


In [ ]:
# 1. Filter the dataset for Gold members only
gold_df = final_df[final_df['membership'] == 'Gold']

# 2. Find the city with the highest total revenue (sum of total_amount) for Gold members
city_revenue = gold_df.groupby('city')['total_amount'].sum()
top_city = city_revenue.idxmax()

# 3. Count how many orders were placed by Gold members in that specific city
order_count = len(gold_df[gold_df['city'] == top_city])

# 4. Display the results
print(f"Top Revenue City for Gold Members: {top_city}")
print(f"Total orders placed by Gold members in {top_city}: {order_count}")

Top Revenue City for Gold Members: Chennai
Total orders placed by Gold members in Chennai: 1337


In [22]:
original_count = len(orders_df)
final_count = len(final_df)

if original_count == final_count:
    print(f"Success! Row count is consistent: {final_count}")
else:
    print(f"Notice: Row count changed from {original_count} to {final_count}")

Success! Row count is consistent: 10000
